In consulting projects, before working on the data, it is crucial to first take a look at what we are provided by the client (ideally we should also show the data to them in the beginning to align).
We should observe the data quality, summarize key inputs (row count, count of unique identifiers) and even do some simple visualizations that would reveal insights that are already there.
Thus, this notebook is to explore the data first. 

In [10]:
import pandas as pd
import os
from pathlib import Path

import constants

Please write the directory you want to work in to "constants.py", and activate the virtual environment there using ".\venv\Scripts\activate" command in shell. Then you will be able to run the notebooks and codes.

In [11]:
base_path_str = constants.BASE_PATH_STR

In [12]:
# define paths
base_path = Path(base_path_str)

data_path = base_path / Path('data')

app_info_path = data_path / Path('applicationInformation.csv')
dev_df_path = data_path / Path('applicationDevelopers.csv')
genre_df_path = data_path / Path('applicationGenres.csv')
publishers_df_path = data_path / Path('applicationPublishers.csv')
languages_df_path = data_path / Path('applicationSupportedlanguages.csv')
count_bottom1000_path = data_path / Path('Playercount_bottom1000.csv')
count_top1000_path = data_path / Path('Playercount_top1000.csv')
price_df_path = data_path / Path('Priceshistory.csv')
tag_df_path = data_path / Path('applicationTags.csv')
packages_df_path = data_path / Path('applicationPackages.csv')

We have a lot of dimension tables and a two fact tables (count tables).

In [5]:
# 1) observe application information
app_info_df = pd.read_csv(app_info_path, encoding='unicode_escape')

In [6]:
app_info_df

,appid,type,name,releasedate,freetoplay
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,21-Dec-17,0.0
1,570,game,Dota 2,9-Jul-13,1.0
2,730,game,Counter-Strike: Global Offensive,21-Aug-12,1.0
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaN,NaN
4,359550,game,Tom Clancy's Rainbow Six Siege,1-Dec-15,0.0
...,...,...,...,...,...
1995,34000,advertising,Football Manager 2010,29-Oct-09,1.0
1996,202480,NaN,Skyrim Creation Kit,NaN,NaN
1997,429050,game,Feed and Grow: Fish,8-Jan-16,0.0
1998,209650,game,Call of Duty: Advanced Warfare,3-Nov-14,0.0


In [7]:
app_info_df.nunique()

appid          2000
type              5
name           1994
releasedate    1130
freetoplay        2
dtype: int64

app_info is our most important/main dataset. We observe that there indeed 2000 unique apps, but unique name count is 1994, so that column might include 
duplicates or NAs. This will be visited in data cleaning phase.

In [10]:
app_info_df[app_info_df.duplicated(['name'], keep=False)].sort_values("name")

,appid,type,name,releasedate,freetoplay
255,12120,game,Grand Theft Auto: San Andreas,6-Jan-11,0.0
868,12250,NaN,Grand Theft Auto: San Andreas,NaN,NaN
194,536930,game,MOBIUS FINAL FANTASY,5-Feb-17,1.0
371,504730,NaN,MOBIUS FINAL FANTASY,NaN,NaN
380,3900,game,Sid Meier's Civilization IV,25-Oct-06,0.0
1364,34440,game,Sid Meier's Civilization IV,25-Oct-06,0.0
236,8800,game,Sid Meier's Civilization IV: Beyond the Sword,24-Jul-07,0.0
1124,34460,game,Sid Meier's Civilization IV: Beyond the Sword,24-Jul-07,0.0
151,323370,game,TERA,5-May-15,1.0
170,212740,NaN,TERA,NaN,NaN


In ETL phase I remove all records having NaNs in either type or releasedate fields, so most of the above should be removed. However, Sid Meier's Civilization IV and Sid Meier's Civilization IV: Beyond the Sword really seem like duplicates. All fields are the same except their appids. We need to omit one of them from each to ensure data integrity. For this I'll keep the first observations later.

In [13]:
# 2) observe the rest
dev_df = pd.read_csv(dev_df_path, encoding='unicode_escape', on_bad_lines='skip', header=None)

# we have a data quality issue with this file, which I overcame by the last argument above.
# we also don't have headers in this file, which I'll assign my data cleaning script.

In [14]:
dev_df

,578080,PUBG Corporation
0,570,Valve
1,622590,NaN
2,359550,Ubisoft Montreal
3,271590,Rockstar North
4,238960,Grinding Gear Games
...,...,...
1788,227860,Sauropod Studio
1789,34000,Sports Interactive
1790,202480,NaN
1791,429050,Old B1ood


In [15]:
genre_df = pd.read_csv(genre_df_path, encoding='unicode_escape', on_bad_lines='skip', header=None)
genre_df

,0,1,2,3
0,578080,Action,Adventure,Massively Multiplayer
1,570,Action,Free to Play,Strategy
2,730,Action,Free to Play,NaN
3,622590,NaN,NaN,NaN
4,359550,Action,NaN,NaN
...,...,...,...,...
1562,227860,Indie,Strategy,NaN
1563,34000,NaN,NaN,NaN
1564,202480,NaN,NaN,NaN
1565,209650,Action,NaN,NaN


As far as I understand, a game can have one, two or three genres assigned to it. If it has only one genre, than the rest of the columns are filled with NaN, which is not a good data design
I will definitely refactor this dataframe into somewhat more interpretable form.

In [46]:
publishers_df = pd.read_csv(publishers_df_path, encoding='unicode_escape', on_bad_lines='skip')
publishers_df

,578080,PUBG Corporation
0,570,Valve
1,730,Valve
2,622590,NaN
3,359550,Ubisoft
4,271590,Rockstar Games
...,...,...
1855,34000,SEGA
1856,202480,NaN
1857,429050,Greens s.r.o.
1858,209650,Activision


We are suffering from missing headers everywhere. Normally we need to check this with client to clarify but for the sake of this assignment I will assign them myself later.

In [47]:
publishers_df.nunique() # --> this verifies that each game has only ONE publisher.

578080              1860
PUBG Corporation     847
dtype: int64

In [48]:
languages_df = pd.read_csv(languages_df_path, encoding='unicode_escape', on_bad_lines='skip')
languages_df

,,,,,,,,,578080,English,Korean,Simplified Chinese,French,German,Spanish - Spain,Arabic,Japanese,Polish,Portuguese,Russian,Turkish,Thai,Italian,Portuguese - Brazil,Traditional Chinese,Ukrainian
570,Bulgarian,Czech,Danish,Dutch,English,Finnish,French,German,Greek,Hungarian,Italian,Japanese,Korean,Norwegian,Polish,Portuguese,Portuguese - Brazil,Romanian,Russian,Simplified Chinese,Spanish - Spain,Swedish,Thai,Traditional Chinese,Turkish,Ukrainian
730,Czech,Danish,Dutch,English,Finnish,French,German,Hungarian,Italian,Japanese,Korean,Norwegian,Polish,Portuguese,Portuguese - Brazil,Romanian,Russian,Simplified Chinese,Spanish - Spain,Swedish,Thai,Traditional Chinese,Turkish,Bulgarian,Ukrainian,NaN
622590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359550,English,French,Italian,German,Spanish - Spain,Czech,Dutch,Japanese,Korean,Polish,Portuguese - Brazil,Russian,Traditional Chinese,Simplified Chinese,Turkish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271590,English,French,Italian,German,Spanish - Spain,Korean,Polish,Portuguese - Brazil,Russian,Traditional Chinese,Japanese,Simplified Chinese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34000,English,Czech,Danish,Dutch,French,Italian,Norwegian,Polish,Portuguese,Spanish - Spain,Swedish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429050,English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209650,English,French,Italian,German,Spanish - Spain,Russian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The same structure with genres dataframe. The languages are listed per column, which is not a good design, I'll visit this.

In [49]:
count_bottom1000 = pd.read_csv(count_bottom1000_path, encoding='unicode_escape', on_bad_lines='skip')
count_bottom1000

,app_id,Time,Playercount
0,20,2017-12-14,44.0
1,20,2017-12-15,49.0
2,20,2017-12-16,59.0
3,20,2017-12-17,63.0
4,20,2017-12-18,53.0
...,...,...,...
972995,757570,2020-08-08,3.0
972996,757570,2020-08-09,4.0
972997,757570,2020-08-10,2.0
972998,757570,2020-08-11,2.0


In [13]:
count_top1000 = pd.read_csv(count_top1000_path, encoding='unicode_escape', on_bad_lines='skip')
count_top1000

,app_id,Time,Playercount
0,10,2017-12-14 00:00:00,8125.0
1,10,2017-12-14 01:00:00,7438.0
2,10,2017-12-14 02:00:00,6695.0
3,10,2017-12-14 03:00:00,6195.0
4,10,2017-12-14 04:00:00,5876.0
...,...,...,...
23351995,9900,2020-08-12 19:00:00,1507.0
23351996,9900,2020-08-12 20:00:00,1473.0
23351997,9900,2020-08-12 21:00:00,1439.0
23351998,9900,2020-08-12 22:00:00,1387.0


These two looks relatively nice. The top one has 23 million rows and my machine with 16 GB RAM might not get along with that, if that's the case I will think of a memory optimizer solution with ready-to-use scripts on internet.

In [16]:
count_top1000[count_top1000.app_id == 244850]

,app_id,Time,Playercount
4787160,244850,2017-12-14 00:00:00,2805.0
4787161,244850,2017-12-14 01:00:00,2940.0
4787162,244850,2017-12-14 02:00:00,2748.0
4787163,244850,2017-12-14 03:00:00,2587.0
4787164,244850,2017-12-14 04:00:00,2364.0
...,...,...,...
4810507,244850,2020-08-12 19:00:00,8556.0
4810508,244850,2020-08-12 20:00:00,8455.0
4810509,244850,2020-08-12 21:00:00,7831.0
4810510,244850,2020-08-12 22:00:00,7278.0


This app's releasedate is 2019-02-28, however it has some player count info from 2017, quite weird, can be a data error.

In fact, I have found out that there are such 13k records. I ran the below query in BigQuery to find out (these tables are created in BigQuery in the later steps of course)

select * from bigquery-dbt-project-baris.xomnia_dataset.t_player_counts_enriched
where release_date > date
;

In [51]:
price_df = pd.read_csv(price_df_path, encoding='unicode_escape', on_bad_lines='skip')
price_df

,app_id,Date,Initialprice,Finalprice,Discount
0,10,2019-04-07,9.99,9.99,0
1,10,2019-04-08,9.99,9.99,0
2,10,2019-04-09,9.99,9.99,0
3,10,2019-04-10,9.99,9.99,0
4,10,2019-04-11,9.99,9.99,0
...,...,...,...,...,...
725263,98800,2020-08-08,4.99,4.99,0
725264,98800,2020-08-09,4.99,4.99,0
725265,98800,2020-08-10,4.99,4.99,0
725266,98800,2020-08-11,4.99,4.99,0


In [52]:
tag_df = pd.read_csv(tag_df_path, encoding='unicode_escape', on_bad_lines='skip')
tag_df

,578080,Survival,Shooter,Multiplayer,PvP,FPS,Third-Person Shooter,Action,Online Co-Op,Tactical,...,Co-op,Early Access,First-Person,Violent,Strategy,Third Person,Competitive,Team-Based,Difficult,Simulation
0,570,Free to Play,MOBA,Strategy,Multiplayer,Team-Based,Action,e-sports,Online Co-Op,Competitive,...,RTS,Difficult,RPG,Fantasy,Tower Defense,Co-op,Character Customization,Replay Value,Action RPG,Simulation
1,730,FPS,Multiplayer,Shooter,Action,Team-Based,Competitive,Tactical,First-Person,e-sports,...,Online Co-Op,Military,Co-op,Strategy,War,Trading,Difficult,Realistic,Fast-Paced,Moddable
2,622590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,359550,FPS,Multiplayer,Tactical,Action,Shooter,Team-Based,First-Person,Co-op,Strategy,...,Online Co-Op,Competitive,Destruction,Atmospheric,Physics,Horror,Singleplayer,Massively Multiplayer,Simulation,Casual
4,271590,Open World,Action,Multiplayer,Third Person,First-Person,Crime,Adventure,Shooter,Third-Person Shooter,...,Racing,Mature,Atmospheric,Sandbox,Co-op,Funny,Great Soundtrack,Comedy,Moddable,RPG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,34000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,202480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,429050,Early Access,Simulation,Underwater,Multiplayer,Action,Survival,Indie,Open World,Singleplayer,...,Colorful,Fishing,Funny,Co-op,Free to Play,NaN,NaN,NaN,NaN,NaN
1997,209650,Dog,FPS,Action,Multiplayer,Shooter,Futuristic,First-Person,Sci-fi,Linear,...,America,War,Quick-Time Events,Short,Illuminati,Online Co-Op,Military,Co-op,PvP,Fast-Paced


In [53]:
packages_df = pd.read_csv(packages_df_path, encoding='unicode_escape', on_bad_lines='skip')
packages_df

,578080,147588,349553
0,570,197846.0,330209.0
1,622590,NaN,NaN
2,238960,306042.0,NaN
3,230410,199291.0,NaN
4,252490,244390.0,NaN
...,...,...,...
1791,227860,26276.0,NaN
1792,34000,NaN,NaN
1793,202480,NaN,NaN
1794,429050,88791.0,NaN
